In [1]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib

from models.convengers import ResNet, Inception, SeaNormaBlock
#from utils import dictionary
from utils import visualizations

plt.ion()   # interactive mode


%load_ext autoreload
%autoreload 2

In [2]:
dataset_folder = "tiny-tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
image_datasets = {"train": datasets.ImageFolder("tiny-tiny-imagenet-200/val", data_transforms['train'])}
dataloaders = {"train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=20, shuffle=True, pin_memory=True)}
dataset_sizes = {"train": len(image_datasets["train"])}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
resnet_new1 = ResNet()
resnet_new2 = ResNet()

inception_new = Inception()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(inception_new.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [15]:
def basic_train(model, optim, criterion, train_loader, num_epochs):
    start_time = time.time()
    for i in range(num_epochs):
        epoch_start_time = time.time()
        train_total, train_correct = 0,0
        for idx, (inputs, targets) in enumerate(train_loader):
            
            optim.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optim.step()
            _, predicted = outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()
            print("\r", end='')
            print(f'training {100 * idx / len(train_loader):.2f}%: {train_correct / train_total:.3f}', end='')
        epoch_end_time = time.time()
        hours, rem = divmod(epoch_end_time-epoch_start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print()
        print("Epoch {} completed with overall accuracy at {:.4f}".format(i, train_correct / train_total))
        print("Epoch {} completed with elapsed time {:0>2}:{:0>2}:{:05.2f}".format(i, int(hours),int(minutes),seconds))
        torch.save({
            'net': model.state_dict(),
        }, 'latest.pt')
    end_time = time.time()
    hours, rem = divmod(end_time-start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Training completed with total elapsed time: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [16]:
basic_train(inception_new, optimizer, criterion, dataloaders["train"], 5)

training 99.80%: 0.086
Epoch 0 completed with overall accuracy at 0.0857
Epoch 0 completed with elapsed time 00:17:03.34
training 99.80%: 0.158
Epoch 1 completed with overall accuracy at 0.1577
Epoch 1 completed with elapsed time 00:17:00.39
training 3.40%: 0.169

KeyboardInterrupt: 

In [17]:
IDX_TO_CLASS = []
with open("./tiny-tiny-imagenet-200/wnids.txt") as f:
    for line in f:
        IDX_TO_CLASS.append(line.strip())
f.close()

CLASS_TO_LABEL = {}
with open("./tiny-tiny-imagenet-200/words.txt") as f:
    for line in f:
        (key, val) = line.split('\t')
        vals = val.strip().split(',')
        CLASS_TO_LABEL[key] = vals[0]
f.close()

def indexToClass(idx):
    return IDX_TO_CLASS[idx]

def classToLabel(class_name):
    return CLASS_TO_LABEL[class_name]

def indexToLabel(idx):
    class_name = indexToClass(idx)
    return classToLabel(class_name)

def num_classes():
    return len(IDX_TO_CLASS)

In [23]:
inputs, classes = next(iter(dataloaders['train']))

class_preds = inception_new(inputs)

In [24]:
print(classes)
print(torch.argmax(class_preds,dim=1))

tensor([139, 167, 123,  84,  94,  84, 147,  77,  53, 185,  56,  43,  30, 188,
        107,  49, 196,   5,  96,  77])
tensor([171,  86,  44,  71,  81, 107, 147,  60, 199,  17,  34,  18,  56,  72,
        129, 108,  14,   5,  96,  60])
